In [2]:
import pyzed.sl as sl
import cv2
import numpy as np

def main():
    # Create a Camera object
    zed = sl.Camera()

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters()
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA
    init_params.coordinate_units = sl.UNIT.METER
    init_params.sdk_verbose = True

    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print("Camera Open : "+repr(err)+". Exit program.")
        exit()

    obj_param = sl.ObjectDetectionParameters()
    obj_param.enable_tracking=True
    obj_param.enable_segmentation=True
    obj_param.detection_model = sl.OBJECT_DETECTION_MODEL.MULTI_CLASS_BOX_ACCURATE

    if obj_param.enable_tracking :
        positional_tracking_param = sl.PositionalTrackingParameters()
        #positional_tracking_param.set_as_static = True
        zed.enable_positional_tracking(positional_tracking_param)

    print("Object Detection: Loading Module...")

    err = zed.enable_object_detection(obj_param)
    if err != sl.ERROR_CODE.SUCCESS :
        print("Enable object detection : "+repr(err)+". Exit program.")
        zed.close()
        exit()

    objects = sl.Objects()
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()
    obj_runtime_param.detection_confidence_threshold = 40
    image = sl.Mat()
    #msk = sl.Mat()
    iter = 0
    while iter < 10:
        zed.grab()
        zed.retrieve_objects(objects, obj_runtime_param)
        if objects.is_new :
            obj_array = objects.object_list
            print(str(len(obj_array))+" Object(s) detected\n")
            if len(obj_array) > 0 :
                zed.retrieve_image(image, sl.VIEW.LEFT)
                image.write(f'output/objdetect/image_{iter:03}imgL.png')
                first_object = obj_array[0]
                print("First object attributes:")
                print(" Label '"+repr(first_object.label)+"' (conf. "+str(int(first_object.confidence))+"/100)")
                if obj_param.enable_tracking :
                    print(" Tracking ID: "+str(int(first_object.id))+" tracking state: "+repr(first_object.tracking_state)+" / "+repr(first_object.action_state))
                position = first_object.position
                velocity = first_object.velocity
                dimensions = first_object.dimensions
                print(" 3D position: [{0},{1},{2}]\n Velocity: [{3},{4},{5}]\n 3D dimentions: [{6},{7},{8}]".format(position[0],position[1],position[2],velocity[0],velocity[1],velocity[2],dimensions[0],dimensions[1],dimensions[2]))
                if first_object.mask.is_init():
                    print(" 2D mask available")
                    first_object.mask.write(f'output/objdetect/mask_{iter:03}img.png')

                print(" Bounding Box 2D ")
                bounding_box_2d = first_object.bounding_box_2d
                for it in bounding_box_2d :
                    print("    "+str(it),end='')
                print("\n Bounding Box 3D ")
                bounding_box = first_object.bounding_box
                for it in bounding_box :
                    print("    "+str(it),end='')

        iter = iter +1

    # Close the camera
    zed.disable_object_detection()
    zed.close()

if __name__ == "__main__":
    main()

Object Detection: Loading Module...
1 Object(s) detected

First object attributes:
 Label 'Electronics' (conf. 60/100)
 Tracking ID: 0 tracking state: OFF / IDLE
 3D position: [0.13777172565460205,-0.08034411072731018,0.6491604447364807]
 Velocity: [nan,nan,nan]
 3D dimentions: [0.10089386999607086,0.13805265724658966,0.10089381784200668]
 Bounding Box 2D 
    [780. 151.]    [928. 151.]    [928. 354.]    [780. 354.]
 Bounding Box 3D 
    [ 0.08756697 -0.15306905  0.60367876]    [ 0.08754236 -0.14600897  0.70432526]    [ 0.18843563 -0.1456573   0.70432526]    [ 0.18846023 -0.15271741  0.60367876]    [ 0.08708699 -0.01535564  0.5940184 ]    [ 0.08706238 -0.00829554  0.6946649 ]    [ 0.18795565 -0.00794389  0.6946649 ]    [ 0.18798026 -0.01500399  0.5940184 ]1 Object(s) detected

First object attributes:
 Label 'Electronics' (conf. 82/100)
 Tracking ID: 0 tracking state: OK / IDLE
 3D position: [0.1380338817834854,-0.08036784082651138,0.6491788029670715]
 Velocity: [0.0036424719728529453,